In [ ]:
# Importamos librerias
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

In [2]:
# Instalación keras-retinanet
!git clone https://github.com/DavidReveloLuna/keras-retinanet.git


fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


In [3]:
cd keras-retinanet/

F:\ProyectoPituca-IA\PitucaDetectionRetN\keras-retinanet


In [4]:
# Instalación y configuración de keras-retinet
!pip install .

Processing f:\proyectopituca-ia\pitucadetectionretn\keras-retinanet
  Using cached opencv_python-4.4.0.46-cp37-cp37m-win_amd64.whl (33.5 MB)
  Using cached progressbar2-3.53.1-py2.py3-none-any.whl (25 kB)
  Using cached python_utils-2.4.0-py2.py3-none-any.whl (12 kB)
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp37-cp37m-win_amd64.whl size=131671 sha256=846a1264c87a73876124eeea94fce6ec4c40fca4f5808599acabb62ea09f3b59
  Stored in directory: c:\users\elias\appdata\local\pip\cache\wheels\4a\bb\3e\bace40a6c40fd8b6d1f2b4b9251137d0d84775dbc62312f074
Successfully built keras-retinanet


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Acceso denegado: 'C:\\Users\\Elias\\anaconda3\\envs\\entornoprueba\\Lib\\site-packages\\cv2\\cv2.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
# Instalación y configuración de keras-retinet
!python setup.py build_ext --inplace


In [ ]:
#importar lo instalado
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [ ]:
cd ..

In [ ]:
# Cargamos el modelo
# debe estar en la carpeta /snapshots/
from keras.models import load_model
from keras_retinanet import models

model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

labels_to_names = pd.read_csv('classes.csv', header=None).T.loc[0].to_dict()


In [ ]:
# Función que realizaz la predicción

import skimage.io as io

def predict(image):
    image = preprocess_image(image.copy())
    image, scale = resize_image(image)

    boxes, scores, labels = model.predict_on_batch(
     np.expand_dims(image, axis=0)
     )

    boxes /= scale

    return boxes, scores, labels


In [ ]:
# Captura de Video
camera = cv2.VideoCapture(0)
camera_height = 500

while(True):

    _, frame = camera.read()
    

    frame = cv2.flip(frame, 1)

    boxes, scores, labels = predict(frame)

    draw = frame.copy()


    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score > 0.8:
            print(box)
            b = box.astype(int)
            color = label_color(label)
            draw_box(draw, b, color=color)
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            

    # show the frame
    cv2.imshow("Test out", draw)


    key = cv2.waitKey(1)

    # quit camera if 'q' key is pressed
    if key & 0xFF == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()